#  Data Analysis: Moral Foundations Theory
---
<img src="https://c1.staticflickr.com/7/6240/6261650491_0cd6c701bb_b.jpg" style="width: 500px; height: 275px;" />

### Professor Amy Tick

Moral Foundations Theory (MFT) hypothesizes that people's sensitivity to the foundations is different based on their political ideology: liberals are more sensitive to care and fairness, while conservatives are equally sensitive to all five. Here, we'll explore whether we can find evidence for MFT in the campaign speeches of 2016 United States presidential candidates. For our main analysis, we'll go through the data science process we learned in Day 1 to recreate a simplified version of the analysis done by Jesse Graham, Jonathan Haidt, and Brian A. Nosek in their 2009 paper ["Liberals and Conservatives Rely on Different Sets of Moral Foundations"](http://projectimplicit.net/nosek/papers/GHN2009.pdf). In part 3, we'll look at other NLP techniques that might be useful in applying this theory.

*Estimated Time: 50 minutes*

---

### Topics Covered
- Plotting data with MatPlotLib
- Interpreting graphs
- Textual analysis methods

### Table of Contents


1 - [Data Set and Test Statistic](#section 1)<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.1 - [2016 Campaign Speeches](#subsection 1)<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.2 - [Moral Foundations Dictionary](#subsection 2) <br>

2 - [Exploratory Data Analysis](#section 2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.1 - [Hypothesis](#subsection 3)<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2 - [Democrats](#subsection 4)<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3 - [Republicans](#subsection 5) <br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.4 - [Democrats vs Republicans](#subsection 6) <br>


**Dependencies:**

In [2]:
from datascience import *
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline
import json



### Before we get started...

Today is going to be a whirlwind tour through the data science process! There will probably be code you don't understand yet, and that's okay. Our goal for today is to show you the ways Data Science can be used in Rhetoric, not to immediately make you into a master programmer. If you have a question at any point (including "I have no idea what's going on"), don't hesitate to ask.

---
## Part 1: Speech Data and Foundations Dictionary  <a id='section 1'></a>

As data scientists starting a new analysis, we know we need to start with two things: some data and a question. In Part 1, we'll get familiar with our data set and determine a way to answer our question using the data.

### 2016 Campaign Speeches <a id='subsection 1'></a>

Our data set is the texts of speeches from the 2016 US presidential campaign. Run the cell below to load the data.

In [16]:
# load the data from csv files into a table. 

speeches = pd.DataFrame()
import os
for file in os.listdir(path='csv'):
    if file.endswith("c.csv"):
        if len(speeches) == 0:
            speeches = pd.read_csv('csv/' + file)
        else:
            speeches = speeches.append(pd.read_csv('csv/' + file))


speeches['Speech'].iloc[80]

'b\'Thank you. Thank you all very, very much. It is wonderful to be with you. I want to thank Gerry Mcentee and Tom Buffenbarger. I want to thank your Congressman Tim Ryan. Also, I want to recognize Representative Ron Gerberry, Representative Linda Bolon, Representative Lorraine Fende, Representative Sandy Harwood. I also want to thank Shelley Murray, the School Board President, and Amy Litkey the Student Rep to the School Board. And I particularly want to thank the Cheney high school and East high school marching bands. Well, hello, Youngstown, how are you tonight? I am thrilled to be here with all of you. And, it is great to see this enthusiasm and this energy. Tonight, I want to talk to you about the choice you have in this election and why that choice matters. It is about picking a president who relies not just on words, but on work, on hard work to get America back to work. That\\\'s our goal. You know, when I think about what we\\\'re really comparing in this election, you know, 

---
Take a moment to look at this table. What information does it contain? What are the different columns? What does each row represent? How large is this table altogether? Hint: there is one Type- 'c' for campaign speech- and two different Parties- 'R' for Republican and 'D' for Democrat.

Answer:

### Moral Foundations Dictionary <a id='subsection 2'></a>

In ["Liberals and Conservatives Rely on Different Sets of Moral Foundations"](http://projectimplicit.net/nosek/papers/GHN2009.pdf), one of the methods Graham, Haidt, and Nosek use to measure people's use of Moral Foundations Theory is to count how often they use words related to each foundation. This will be our test statistic for today. To calculate it, we'll need a dictionary of words related to each moral foundation. 

The dictionary we'll use today comes from a database called [WordNet](https://wordnet.princeton.edu), in which "nouns, verbs, adjectives and adverbs are grouped into sets of cognitive synonyms (synsets), each expressing a distinct concept." By querying WordNet for semantically related words, it was possible to build a dictionary automatically using a Python program.

Run the cell below to load the dictionary and assign it to the variable 'mft_dict'.

In [8]:
# Run this cell to load the dictionary into a variable
with open('foundations_dict.json') as json_data:
    mft_dict = json.load(json_data)


We can see the keys of the dictionary using the .keys() function:

In [9]:
mft_dict.keys()

dict_keys(['authority/respect', 'care', 'fairness/proportionality', 'liberty', 'loyalty/ingroup', 'sanctity/purity'])

And we can look up the entries associated with a key by putting the key in brackets:

In [10]:
mft_dict['authority/respect']

['author',
 'authoris',
 'domin',
 'say-so',
 'assur',
 'confid',
 'sure',
 'agenc',
 'federal ag',
 'government ag',
 'bureau',
 'offic',
 'sanction',
 'respect',
 'regard',
 'esteem',
 'defer',
 'obedi',
 'valu',
 'honor',
 'honour',
 'abide bi',
 'observ']

Try looking up the entries for the other keys by filling in for '...' in the cell below.

In [11]:
mft_dict['...']

KeyError: '...'

There's something odd about some of the entries: they're not words! The entries in this dictionary have been **stemmed**, meaning they have been reduced to their smallest meaningful root. 

We can see why this is helpful with an example. Python can count the number of times a string can be found in another string using the string method 'count':

In [ ]:
# Counts the number of times the second string appears in the first string
"Data science is the best major, says data scientist.".count('science')

It returns one match, for the second word. But, 'scientist' is very closely related to 'science', and many times we will want to match them both. A stem allows Python to find all words with a common root. Try running the count again with a stem that matches both 'science' and 'scientist'.

In [ ]:
# Fill in the parenthesis with a stem that will match both 'science' and 'scientist'
"Data science is the best major, says data scientist.".count('...')

Another thing you might have noticed is that all the entries in our dictionary are lowercase. This could be a problem when we do our text analysis. Try counting the number of times 'rhetoric' appears in the example sentence.

In [ ]:
# Fill in the parenthesis to count how often 'rhetoric' appears in the sentence
"Rhetoric major says back: NEVER argue with a rhetoric student.".count('...')

We can clearly see the word 'rhetoric' appears twice, but the count function only returns 1. That's because Python differentiates between capital and lowercase letters:

In [ ]:
'r' is 'R'

To get around this, we can use the .lower() function, which changes all letters in the string to lowercase:

In [ ]:
"Rhetoric major says back: NEVER argue with a rhetoric student.".lower()

Let's add a column to our 'speeches' table that contains the lowercase text of the speeches.

In [12]:
def clean_text(text):
    # remove punctuation
    p = re.compile(r'[^\w\s]')
    no_punc = p.sub(' ', text)
    # convert to lowercase
    no_punc_lower = no_punc.lower()
    # split into individual words
    return no_punc_lower
    
speeches['clean_speech'] = [clean_text(s) for s in speeches['Speech']]

speeches.head()

,Candidate,Party,Type,Date,Title,Speech,clean_speech
0,Jeb Bush,R,c,"June 15, 2015",b'Remarks Announcing Candidacy for President a...,b'Thank you all very much. I always feel welco...,b thank you all very much i always feel welco...
1,Jeb Bush,R,c,"July 30, 2015",b'Remarks to the National Urban League Confere...,"b""Thank you all very much. I appreciate your h...",b thank you all very much i appreciate your h...
2,Jeb Bush,R,c,"August 11, 2015",b'Remarks at the Ronald Reagan Presidential Li...,"b""Thank you very much. It's good to be with al...",b thank you very much it s good to be with al...
3,Jeb Bush,R,c,"September 9, 2015","b'Remarks in Garner, North Carolina'",b'Thank you very much. I appreciate your hospi...,b thank you very much i appreciate your hospi...
4,Jeb Bush,R,c,"November 2, 2015","b'Remarks in Tampa, Florida'",b'Thank you. It\'s great to be in Tampa with s...,b thank you it s great to be in tampa with s...


What might be some pros or cons to their method versus the method used for our dictionary?

Answer:

---
## Part 2: Exploratory Data Analysis <a id='section 2'></a>

Now that we have our speech data and our dictionary, we can start our analysis. First, we'll formally state our hypothesis. Then, to visualize the data we'll perform 3 steps:
1. Count the occurances of words from our dictionary in each speech
2. Calculate how often words from each category are used by each political party
3. Plot the percents on a bar graph

### Hypothesis <a id='subsection 3'></a>

An important part of data science is understanding the question you're trying to answer and formulating an appropriate hypothesis. The hypothesis must be testable given your data, and you must be able to say what kinds of results would support or refute your hypothesis. 

Today, our question asks whether the word use of 2016 presidential candidates aligns with Moral Foundations Theory.

Think about what you know about Moral Foundations Theory. If this data is consistent with the theory, what should our analysis show for Republican candidates? What about for Democratic candidates? Try sketching a possible graph for each political party, assuming that candidates' speech aligns with the theory.

In [ ]:
# answer

### Democrats <a id='subsection 4'></a>

Let's start by looking at Democratic candidates. First, we need to make a table that only contains Democrats. Run the cell below to do so.

In [13]:
# Filter out non-Democrat speeches
democrats = speeches[speeches['Party'] == 'D']
democrats.head()

,Candidate,Party,Type,Date,Title,Speech,clean_speech
0,Lincoln Chafee,D,c,"June 3, 2015",b'Remarks Announcing Candidacy for President a...,"b'Thank you, Bob. Thank you, Bob and Mark, ver...",b thank you bob thank you bob and mark ver...
1,Lincoln Chafee,D,c,"July 17, 2015",b'Remarks at the Iowa Democrats Hall of Fame D...,b'Congratulations to the Hall of Fame Inductee...,b congratulations to the hall of fame inductee...
2,Lincoln Chafee,D,c,"October 23, 2015",b'Remarks Announcing the End of Presidential C...,b'Once again it is a pleasure to join so many ...,b once again it is a pleasure to join so many ...
0,Hillary Clinton,D,c,"January 20, 2007",b'Video Transcript: Presidential Exploratory C...,"b""HILLARY CLINTON: I announced today that I am...",b hillary clinton i announced today that i am...
1,Hillary Clinton,D,c,"January 22, 2007","b'Remarks in a ""Let the Conversation Begin Web...","b'SENATOR CLINTON: Hi, everyone, and welcome t...",b senator clinton hi everyone and welcome t...


Our test statistic is the percent of words that correspond to a Moral Foundation in Democratic speeches- in other words, how often candidates use words related to a specific foundation. 

(Bonus question: why don't we just use the **number** of Moral Foundation words instead of the **percent** as our test statistic?)

To calculate the percent, we'll first need the total number of words in each speech.

In [3]:
democrats['total_words'] = [len(speech.split()) for speech in democrats['Speech']]
democrats.head()

NameError: name 'democrats' is not defined

Next, we need to calculate the number of matches to entries in our dictionary for each speech and for each foundation.

In [15]:
for key in mft_dict.keys():
    num_key_words = np.zeros(len(democrats))
    synonyms = mft_dict[key]
    for synonym in synonyms:
        syn_count = np.array([sum([wd.startswith(synonym) for wd in speech.split()]) for speech in democrats['clean_speech']])
        num_key_words += syn_count
    democrats[key] = num_key_words / democrats['total_words'] * 100

democrats.head()

C:\Users\keeley\AppData\Local\conda\conda\envs\dsmodules\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Candidate,Party,Type,Date,Title,Speech,clean_speech,total_words,authority/respect,care,fairness/proportionality,liberty,loyalty/ingroup,sanctity/purity
0,Lincoln Chafee,D,c,"June 3, 2015",b'Remarks Announcing Candidacy for President a...,"b'Thank you, Bob. Thank you, Bob and Mark, ver...",b thank you bob thank you bob and mark ver...,5264,0.002090,0.001710,0.002090,0.000000,0.002090,0.000000
1,Lincoln Chafee,D,c,"July 17, 2015",b'Remarks at the Iowa Democrats Hall of Fame D...,b'Congratulations to the Hall of Fame Inductee...,b congratulations to the hall of fame inductee...,700,0.002857,0.001429,0.002857,0.000000,0.004286,0.000000
2,Lincoln Chafee,D,c,"October 23, 2015",b'Remarks Announcing the End of Presidential C...,b'Once again it is a pleasure to join so many ...,b once again it is a pleasure to join so many ...,902,0.002217,0.001109,0.002217,0.001109,0.005543,0.000000
0,Hillary Clinton,D,c,"January 20, 2007",b'Video Transcript: Presidential Exploratory C...,"b""HILLARY CLINTON: I announced today that I am...",b hillary clinton i announced today that i am...,330,0.003030,0.006061,0.000000,0.000000,0.009091,0.000000
1,Hillary Clinton,D,c,"January 22, 2007","b'Remarks in a ""Let the Conversation Begin Web...","b'SENATOR CLINTON: Hi, everyone, and welcome t...",b senator clinton hi everyone and welcome t...,5158,0.003296,0.002714,0.000969,0.000000,0.002520,0.000582


We have our proportions, but it's much easier to understand what's going on when the results are in graph form. Let's start by looking at the average proportions for Democrats as a group. Run the cell below to show a graph of the average proportions. Again, don't worry about the details of the code.

In [ ]:
avg_dem_stats = Table().with_columns('Moral Foundation', mft_dict.keys(),
                                    'Proportion', [np.average(democrats[mf]) for mf in mft_dict.keys()])
avg_dem_stats.barh('Moral Foundation')

Take a look at this graph. What does it show? Does it support our hypothesis?

Answer:

We can also look at how different candidates used different foundations.

In [ ]:
dem_indivs = (democrats.loc[:, ['Candidate', 'authority/respect', 'care', 'loyalty/ingroup', 'fairness/proportionality',
                               'sanctity/purity', 'liberty']]
             .groupby(['Candidate'])
             .mean())
dem_indivs.plot.bar(figsize=(10, 8))

### Republicans <a id='subsection 5'></a>

Now, let's repeat the process for Republicans. Replace the ellipses with the correct code to select only Republican speeches, then run the cell to create the table. 

(Hint: look back at how we made the 'democrats' table to see how to fill in the ellipses)

In [16]:
# Filter out non-Republican speeches
republicans = speeches[speeches['Party'] == 'R']
republicans.head()

,Candidate,Party,Type,Date,Title,Speech,clean_speech
0,Jeb Bush,R,c,"June 15, 2015",b'Remarks Announcing Candidacy for President a...,b'Thank you all very much. I always feel welco...,b thank you all very much i always feel welco...
1,Jeb Bush,R,c,"July 30, 2015",b'Remarks to the National Urban League Confere...,"b""Thank you all very much. I appreciate your h...",b thank you all very much i appreciate your h...
2,Jeb Bush,R,c,"August 11, 2015",b'Remarks at the Ronald Reagan Presidential Li...,"b""Thank you very much. It's good to be with al...",b thank you very much it s good to be with al...
3,Jeb Bush,R,c,"September 9, 2015","b'Remarks in Garner, North Carolina'",b'Thank you very much. I appreciate your hospi...,b thank you very much i appreciate your hospi...
4,Jeb Bush,R,c,"November 2, 2015","b'Remarks in Tampa, Florida'",b'Thank you. It\'s great to be in Tampa with s...,b thank you it s great to be in tampa with s...


Next, we need to calculate our test statistic for Republicans. Fill in the ellipses in the cell below with the correct code to create a table with the statistics. Once again, look at how we made this table for Democrats, and think about how you need to change the code for Republicans.

In [17]:
# Calculate the proportions for Republican speeches
republicans['total_words'] = [len(speech.split()) for speech in republicans['Speech']]
republicans.head()

C:\Users\keeley\AppData\Local\conda\conda\envs\dsmodules\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Candidate,Party,Type,Date,Title,Speech,clean_speech,total_words
0,Jeb Bush,R,c,"June 15, 2015",b'Remarks Announcing Candidacy for President a...,b'Thank you all very much. I always feel welco...,b thank you all very much i always feel welco...,2126
1,Jeb Bush,R,c,"July 30, 2015",b'Remarks to the National Urban League Confere...,"b""Thank you all very much. I appreciate your h...",b thank you all very much i appreciate your h...,2528
2,Jeb Bush,R,c,"August 11, 2015",b'Remarks at the Ronald Reagan Presidential Li...,"b""Thank you very much. It's good to be with al...",b thank you very much it s good to be with al...,3624
3,Jeb Bush,R,c,"September 9, 2015","b'Remarks in Garner, North Carolina'",b'Thank you very much. I appreciate your hospi...,b thank you very much i appreciate your hospi...,1785
4,Jeb Bush,R,c,"November 2, 2015","b'Remarks in Tampa, Florida'",b'Thank you. It\'s great to be in Tampa with s...,b thank you it s great to be in tampa with s...,2318


Then, calculate foundation synonym percents:

In [18]:
for key in mft_dict.keys():
    num_key_words = np.zeros(len(republicans))
    synonyms = mft_dict[key]
    for synonym in synonyms:
        syn_count = np.array([sum([wd.startswith(synonym) for wd in speech.split()]) for speech in republicans['clean_speech']])
        num_key_words += syn_count
    republicans[key] = num_key_words / republicans['total_words'] * 100
    
republicans.head()

C:\Users\keeley\AppData\Local\conda\conda\envs\dsmodules\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Candidate,Party,Type,Date,Title,Speech,clean_speech,total_words,authority/respect,care,fairness/proportionality,liberty,loyalty/ingroup,sanctity/purity
0,Jeb Bush,R,c,"June 15, 2015",b'Remarks Announcing Candidacy for President a...,b'Thank you all very much. I always feel welco...,b thank you all very much i always feel welco...,2126,0.001881,0.002352,0.003293,0.0,0.003293,0.000941
1,Jeb Bush,R,c,"July 30, 2015",b'Remarks to the National Urban League Confere...,"b""Thank you all very much. I appreciate your h...",b thank you all very much i appreciate your h...,2528,0.003165,0.000791,0.002769,0.0,0.000791,0.000791
2,Jeb Bush,R,c,"August 11, 2015",b'Remarks at the Ronald Reagan Presidential Li...,"b""Thank you very much. It's good to be with al...",b thank you very much it s good to be with al...,3624,0.003311,0.001380,0.001656,0.0,0.000552,0.000828
3,Jeb Bush,R,c,"September 9, 2015","b'Remarks in Garner, North Carolina'",b'Thank you very much. I appreciate your hospi...,b thank you very much i appreciate your hospi...,1785,0.001681,0.000560,0.001681,0.0,0.000000,0.000000
4,Jeb Bush,R,c,"November 2, 2015","b'Remarks in Tampa, Florida'",b'Thank you. It\'s great to be in Tampa with s...,b thank you it s great to be in tampa with s...,2318,0.003020,0.001294,0.000431,0.0,0.001294,0.001294


Then, run the next cell to show a graph of the average Republican percentages.

In [ ]:
avg_rep_stats = Table().with_columns('Moral Foundation', mft_dict.keys(),
                                    'Proportion', [np.mean(republicans[mf]) for mf in mft_dict.keys()])
avg_rep_stats.barh('Moral Foundation')

Does this graph support our hypothesis? 
Answer:

Finally, let's look at individual Republican candidate averages.

In [ ]:
rep_indivs = (republicans.loc[:, ['Candidate', 'authority/respect', 'care', 'loyalty/ingroup', 'fairness/proportionality',
                               'sanctity/purity', 'liberty']]
             .groupby(['Candidate'])
             .mean())
rep_indivs.plot.bar(figsize=(15, 8))

### Democrats vs Republicans <a id='subsection 6'></a>

Comparing two groups becomes much easier when we can look at them both at the same time. Run the cell below to get a graph for side-by-side comparison.

In [ ]:
all_avg_stats = Table().with_columns('Moral Foundation', mft_dict.keys(),
                                    'Democrats', [np.mean(democrats[mf]) for mf in mft_dict.keys()],
                                    'Republicans', [np.mean(republicans[mf]) for mf in mft_dict.keys()])
all_avg_stats.barh('Moral Foundation')

We can also compare the stats for the Democratic and Republican nominees.

In [ ]:
all_avg_stats = Table().with_columns('Moral Foundation', mft_dict.keys(),
                                    'Hillary Clinton', dem_indivs.loc['Hillary Clinton ', :],
                                    'Donald Trump', rep_indivs.loc['Donald Trump', :])
all_avg_stats.barh('Moral Foundation')

---

## Bibliography

* Election documents scraped from http://www.presidency.ucsb.edu/2016_election.php
* Graham, J., Haidt, J., & Nosek, B. A. (2009). Liberals and conservatives rely on different sets of moral foundations. Journal of personality and social psychology, 96(5), 1029. http://projectimplicit.net/nosek/papers/GHN2009.pdf, October 9 2017.

---
Notebook developed by: Keeley Takimoto, Sean Seungwoo Son, Sujude Dalieh

Data Science Modules: http://data.berkeley.edu/education/modules
